In [1]:
# test of folium
import folium
from folium import IFrame
# dir(folium)


In [2]:
home = [42.8179954, -73.943261]
map = folium.Map(location=home, zoom_start=6)
map.add_child(folium.Marker(location=home, popup="Home", icon=folium.Icon(color='blue')))

In [3]:
map.save("Map1.html")



In [4]:
import csv
# set up csv reader dialect to accept quoted strings with comma seperator
csv.register_dialect('myDialect',
delimiter = ',',
quoting=csv.QUOTE_ALL,
skipinitialspace=True)



In [5]:
# read zip code to latitude longitude table

with open('ziplatlon.csv', 'r') as f:
    reader = csv.reader(f, dialect='myDialect')
    ziptable = {}
    for row in reader:
        ziptable[row[0]] = [row[1], row[2]]

print(ziptable["12305"])

['42.811981', '-73.950252']


In [6]:
maxi = 0
with open('residencehallhate151617.csv', 'r') as f:
    rh_reader = csv.reader(f, dialect='myDialect')
    rhh = {}
    for row in rh_reader:
        if row[0] == "UNITID_P":
            rhh_schema = row
            rhh_schema.append("lat")
            rhh_schema.append("lon")
            rhh_schema.append("isum")
            idx = 0
            rhh_si = {}
            rhh_rsi = {}
            for item in rhh_schema:
                rhh_si[item] = idx
                rhh_rsi[idx] = item
                idx = idx + 1
                
        else:
            if row[0] in rhh:
                print(row[0], "already exists")
            else:
                zip = row[6][0:5]
                if zip in ziptable:
                    lat = ziptable[zip][0]
                    lon = ziptable[zip][1]
                else:
                    lat = 0
                    lon = 0
                # calculate and append the sum of incidents
                isum = 0
                for idx in range(12,355):
                    try:
                        isum = isum + int(row[idx])
                    except ValueError:
                        isum = isum
                row.append(lat)
                row.append(lon)
                row.append(isum) 
                if isum > maxi:
                    maxi = isum
                rhh[row[0]] = row
              #  print(zip)
    
    uc = rhh["196866001"]
#    print(uc)
#    zip = uc[6][0:5]
#    print(zip)
#    print(rhh_si)
#    print(uc[rhh_si["lat"]],uc[rhh_si["lon"]])

print("max incidents:",maxi)

max incidents: 42


In [7]:
# save the rhh, schema, reverse schema
import pickle

afile = open(r'rhh.pkl', 'wb')
pickle.dump(rhh, afile)
afile.close()

afile = open(r'rhh_si.pkl', 'wb')
pickle.dump(rhh_si, afile)
afile.close()

afile = open(r'rhh_rsi.pkl', 'wb')
pickle.dump(rhh_rsi, afile)
afile.close()

#reload object from file
#file2 = open(r'C:\d.pkl', 'rb')
#new_d = pickle.load(file2)
#file2.close()


In [8]:
def rhhdump(r):
    a = "<b>" + str(r[rhh_si["isum"]]) + " incidents</b><br><br>\n"
    for i in sorted(rhh_rsi):
        k = rhh_rsi[i]
        v = r[i]
        if i > 11:
            try:
                v = int(r[i])
            except ValueError:
                try:
                    v = float(r[i])
                except ValueError:
                    v = ""
        if (int(i) < 12) or v:
            a = a + k + ": " + str(v) + "<br>\n"
    return a

da = {}
trouble = 0
for inst in rhh:
    uc = rhh[inst]
    isum = uc[rhh_si["isum"]]
    if isum in da:
        da[isum] = da[isum] + 1
    else:
        da[isum] = 1
    loc = [float(uc[rhh_si["lat"]]),float(uc[rhh_si["lon"]])]
    po = uc[1] + " " + uc[2] + " " + str(isum)
    icolor = "green"
    if isum > 7:
        if isum > 15:
            if isum > 20:
                text = rhhdump(uc) + '<a href="https://www.albany.edu">Visit UAlbany</a>'
                iframe = folium.IFrame(text, width=350, height=150)
                popup = folium.Popup(iframe, max_width=3000)
                Text = folium.Marker(location=loc, popup=popup,
                                     icon=folium.Icon(icon_color='darkred'))
                map.add_child(Text)
                print(uc[1],uc[2],uc[3],uc[4],uc[5],loc)
                icolor = "darkred"
                trouble += 1
                folium.Circle(
                    radius=10000,
                    location=loc,
                    popup='Danger',
                    color='crimson',
                    fill=False,).add_to(map)
            else:
                icolor = "lightred"
        else:
            icolor = "orange"
            
    if (isum > 5) and (isum < 20):
#        print(icolor)
        map.add_child(folium.Marker(location=loc, popup=po, icon=folium.Icon(color=icolor)))

    

Keene State College Keene State College 229 Main Street Keene NH [0.0, 0.0]
Onondaga Community College Main Campus 4585 West Seneca Turnpike Syracuse NY [42.980793, -76.223386]
Ohio State University-Main Campus Main Campus 190 N. OVAL MALL COLUMBUS OH [40.005435, -83.023227]


In [9]:

map.save("Map1.html")
print(da)
print("trouble",trouble)

{0: 10849, 2: 184, 4: 61, 8: 13, 10: 10, 6: 36, 14: 2, 9: 1, 16: 3, 1: 4, 3: 5, 11: 2, 42: 1, 12: 5, 18: 1, 24: 1, 25: 1, 15: 2}
trouble 3


In [10]:
# experiment with json encoding
import json
uc = rhh["196866001"]
ua = rhh["196060001"]
r = {"196866001": uc, "196060001": ua}
jd = json.dumps(r)
print(jd)


{"196866001": ["196866001", "Union College", "Main Campus", "807 UNION ST", "SCHENECTADY", "NY", "123082311", "2", "Private nonprofit, 4-year or above", "1219", "1048", "2267", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0", "0"

In [11]:
jl = json.loads(jd)
print(jl)

{'196866001': ['196866001', 'Union College', 'Main Campus', '807 UNION ST', 'SCHENECTADY', 'NY', '123082311', '2', 'Private nonprofit, 4-year or above', '1219', '1048', '2267', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0', '0'